# タスク 2a: Anthropic Claude を使用した小さなファイルによるテキスト要約

このノートブックでは、小さなテキスト文字列を Amazon Bedrock API (Anthropic Claude モデルを使用) に直接取り込み、入力テキストを要約するように指示します。入力テキストの長さがモデルのコンテキストサイズ制限内であれば、このアプローチを適用して、通話記録、会議記録、書籍、記事、ブログ投稿、その他の関連コンテンツを要約できます。


## タスク 2a.1: 環境のセットアップ

このタスクでは、環境をセットアップします。

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## タスク 2a.2: 要約するテキストを含むプロンプトの作成

このタスクでは、基盤モデルでサポートされている最大長よりも少ないトークンを含む短いテキストを使用します。このラボのサンプル入力テキストとして、Amazon Bedrock を発表する [AWS ブログ投稿](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) の段落を日本語に翻訳したものを使用します。

プロンプトは、 `次のテキストの要約を入力してください。` という指示で始まります。

In [ ]:
prompt_data = """

次のテキストの要約を入力してください:

AWS はお客様からのフィードバックをすべて取り入れ、本日、AI21 Labs、Anthropic、Stability AI、Amazon の FM に API 経由でアクセスできるようにする新しいサービスである Amazon Bedrock を発表できることを嬉しく思います。
Bedrock は、FM を使用して AI ベースの生成アプリケーションを構築および拡張する最も簡単な方法であり、すべてのビルダーがアクセスできるようになります。
Bedrock は、スケーラブルで信頼性が高く、安全な AWS マネージドサービスを通じて、テキストと画像用のさまざまな強力な FM (本日発表する 2 つの新しい LLM で構成される Amazon の Titan FM を含む) にアクセスする機能を提供します。 
Bedrock のサーバーレス エクスペリエンスにより、お客様は、実行しようとしていることに適したモデルを簡単に見つけ、すぐに開始し、独自のデータを使用して FM をプライベートにカスタマイズし、使い慣れた AWS ツールと機能を使用してアプリケーションに簡単に統合してデプロイできます。
インフラストラクチャを管理する必要はありません
(さまざまなモデルをテストするための Experiments や、FM を大規模に管理するための Pipelines などの Amazon SageMaker ML 機能との統合を含む)。
"""

## タスク 2a.3: プロンプトと推論パラメータを含むリクエスト本文の作成

このタスクでは、上記のプロンプトと推論パラメータを含むリクエスト本文を作成します。

In [ ]:
# request body
body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 2048,
    "temperature": 0,
    "top_p": 0.9,
    "messages": [
     {
            "role": "user",
            "content": [{"type": "text", "text": prompt_data}],
     }
    ],
})

## タスク 2a.4: Boto3 経由で基盤モデルを呼び出す

このタスクでは、リクエストパラメータ `modelId`、`accept`、および `contentType` を指定して、Amazon Bedrock に API リクエストを送信します。提供されたプロンプトに続いて、Amazon Bedrock の基盤モデルが入力テキストを要約します。

### 完全な出力生成

デフォルトでは、Amazon Bedrock サービスは、特定のプロンプトの要約全体を 1 つの出力で生成します。モデル出力に多くのトークンが含まれている場合、出力する速度が遅くなる可能性があります。

In [ ]:
#model configuration and invoke the model
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body["content"][0]["text"]

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

print(outputText)

### ストリーミング出力生成

次に、Amazon Bedrock のinvoke_model_with_response_stream API を使用してモデル出力をストリーミングし、ユーザーが生成された出力を消費できるようにする方法を探索します。この API は、完全な出力を一度に生成するのではなく、モデルから生成される小さな出力チャンクを送信する ResponseStream を返します。これらのストリーミング出力は、連続した消費可能なビューで表示できます。

In [ ]:
#invoke model with response stream
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')

#output = list(stream)

# Collect only the first few chunks
output = []
for i, event in enumerate(stream):
    output.append(event)
    if i >= 4:  # Show only first 5 chunks
        print(f"... and {i} more chunks (truncated)")
        break

# Display truncated output
output

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            if chunk_obj["type"] == "content_block_delta":
                    text = chunk_obj["delta"].get("text", "")
                    clear_output(wait=True)
                    output.append(text)
                    display_markdown(Markdown(''.join(output)))
                    i+=1


これで、boto3 SDK を使用して Amazon Bedrock API にアクセスする実験が完了しました。この SDK は、Bedrock 機能への基本的なプログラムによるアクセスを提供します。この API を活用することで、1) AWS ニュースコンテンツのテキスト要約全体を一度に生成する、2) 要約出力をチャンクでストリーミングして増分処理するという 2 つのユースケースを実装できました。

### 試してみましょう
- 特定のユースケースに合わせてプロンプトを変更し、さまざまなモデルの出力を評価します。
- トークンの長さを変えることで、サービスのレイテンシと応答性がどのように変化するかを理解します。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を取得します。

### クリーンアップ

あなたはこのノートブックを完了しました。ラボの次のパートに移るには、下記を実行してください。:

- このノートブックファイルを閉じ、 **Task2b_ja_jp.ipynb** を開きます。
